# Contexto

Uma página da web foi reformulada devido a uma baixa taxa de conversão de vendas entre os visitantes. Para avaliar a eficácia da nova versão da página, foram criados dois grupos de controle distintos. O objetivo é medir o quanto a página remodelada pode ser mais atraente para os clientes e incentivar a compra durante a visita. Um conjunto de dados foi disponibilizado para ajudar a analisar o impacto da nova página.

Hₒ: p = pₒ

Hₐ: p ≠ pₒ

Também definiremos um nível de confiança de 95%

Escolhendo as variáveis.
Para nosso teste, temos dois grupos:

Um grupo de controle - Eles viram o design antigo

Um grupo de tratamento (ou experimental) - Eles viram o novo design

Coluna converted 

0 - O usuário não comprou o produto durante esta sessão de usuário

1 - O usuário comprou o produto durante esta sessão de usuário

# Bibliotecas

In [11]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

# Dataset

In [9]:
df = pd.read_csv(r'C:\Users\Andre\EBA\Aula-4\bases\df_test.csv')

In [12]:
df

,Unnamed: 0,user_id,timestamp,landing_page,converted,group
0,281655,918193,2017-01-03 13:42:27.285433,new_page,1,treatment
1,7271,679658,2017-01-20 02:00:54.448676,new_page,1,treatment
2,269760,815158,2017-01-16 05:11:33.823264,new_page,1,treatment
3,158882,857307,2017-01-22 08:58:50.687626,new_page,1,treatment
4,153958,716896,2017-01-15 17:59:03.065352,new_page,1,treatment
...,...,...,...,...,...,...
5631,2813,848867,2017-01-03 04:37:58.815088,old_page,0,control
5632,2814,896519,2017-01-05 18:54:30.314378,old_page,0,control
5633,2815,639731,2017-01-23 10:15:31.261524,old_page,0,control
5634,2816,788990,2017-01-10 12:51:44.488371,old_page,0,control


Vamos supor que o dataset esteja totalmente limpo, sem dados duplicados e a base pronta para trabalhar

# A partir dos testes de hipótese, vamos comprovar se com o novo layout houve mudança no comportamento do usuário...

Filtrando o df e criando dois diferentes, um apenas com os dados do grupo de teste (treatment) e outro com os de controle (control)

In [13]:

grupo_tratamento = df[df.group == "treatment"]
grupo_controle = df[df.group == "control"]

Calculando a taxa de conversão para cada grupo (conversão = numero de pessoas que compraram o produto / total de pessoas do grupo)

In [14]:
conversao_tratamento = grupo_tratamento[grupo_tratamento.converted==1].user_id.count()/grupo_tratamento.user_id.count()

conversao_controle = grupo_controle[grupo_controle.converted==1].user_id.count()/grupo_controle.user_id.count()

print(conversao_tratamento, conversao_controle)

0.18985095812633074 0.12455642299503193


Para o grupo tratamento tivemos uma taxa de conversão de 18.9% e para o controle 12.4%.

Vimos aqui que, aparentemente, houve melhora na média. Mas será que é estatisticamente significativa?

A depender do que você tem em sua amostra, escolhemos o teste a ser usado. Aqui estamos falando de uma proporção (conversão é a proporção) e, portanto, vamos usar o teste de comparação de proporções.

In [15]:
# importando a biblioteca para tirar o ztest
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [16]:
# definimos os grupos de controle/tratamento
control_results = grupo_controle['converted']
treatment_results = grupo_tratamento['converted']

#definimos o numero sucesso que tivemos em cada grupo, ou seja, quantos 1 tivemos em cada grupo

successes = [control_results.sum(), treatment_results.sum()]

# calculamos o tamanho da amostra de cada grupo (tamanho será o numero total de registros de cada grupo)
n_con = grupo_controle.user_id.count()
n_treat = grupo_tratamento.user_id.count()
nobs = [n_con, n_treat] #tamanho das amostras de cada grupo

z_stat,pvalue = proportions_ztest(successes, nobs=nobs) #calculamos z para o teste e seu respectivo pvalor
print(z_stat, pvalue)

-6.733481162342496 1.6565111652777357e-11


Nosso teste era bilateral (pois nossas hipoteses são Hₒ: p = pₒ e Hₐ: p ≠ pₒ). Lembrando que para esse caso, a zona de rejeição é z<-1.96 e z>1.96. Logo, para z=-6.73 REJEITAMOS a hipótese nula. Ou seja, as proporções não são iguais.

Podemos rejeitar também pelo valor menor que 0.05 do pvalue

In [17]:
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)


print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

ci 95% for control group: [0.112, 0.137]
ci 95% for treatment group: [0.175, 0.204]


Para decidir se temos evidencias estatisticamente significativas para rejeitar a hipótese nula, temos de avaliar os seguinte pontos:

Se os Intervalos de Confiança Não se Sobrepõem:

- Se os intervalos de confiança para as proporções de sucesso dos grupos de controle e tratamento não se sobrepõem, isso sugere que as proporções de sucesso são significativamente diferentes entre os grupos.

Se os Intervalos de Confiança se Sobrepõem:

- Você pode não ter evidências suficientes para concluir que houve uma diferença significativa na conversão entre os grupos apenas com base nos intervalos de confiança.
- No nosso caso, os intervalos NÃO se sobrepoem logo isso sugere que, com 95% de confiança, há uma diferença significativa nas proporções de sucesso entre os grupos de controle e tratamento.

